In [14]:
import pandas as pd
import glob
import os

ruta_carpeta_911 = "../data/raw/ecu911/dataset/"
ruta_catalogo = "../data/processed/"
ruta_dest = "../data/processed/ecu911"

archivos_csv = glob.glob(os.path.join(ruta_carpeta_911, "*.csv"))

In [15]:
lista_dfs = []

for archivo in archivos_csv:
    df_temp = pd.read_csv(
        archivo,
        sep=";",
        encoding="UTF-8",
        dtype={"Cod_Parroquia": str},
        on_bad_lines="skip"
    )

    df_temp.columns = (
        df_temp.columns.astype(str)
        .str.replace("ï»¿", "")
        .str.strip()
        .str.lower()
    )

    if "servicio" in df_temp.columns:
        df_temp = df_temp[df_temp["servicio"] == "Seguridad Ciudadana"]

    lista_dfs.append(df_temp)

In [16]:
df_911 = pd.concat(lista_dfs, ignore_index=True)

In [17]:
# Fecha
df_911["fecha_dt"] = pd.to_datetime(df_911["fecha"], errors="coerce", dayfirst=True)
df_911 = df_911.dropna(subset=["fecha_dt"])


In [18]:
# Normalizar cod_parroquia
df_911["cod_parroquia"] = (
    df_911["cod_parroquia"]
    .astype(str)
    .str.replace(".0", "", regex=False)
    .str.zfill(6)
)

In [19]:
# (CLAVE) agrupar por parroquia y día (si quieres día, normaliza a date)
df_911["fecha_dia"] = df_911["fecha_dt"].dt.date

df_group = (
    df_911.groupby(["cod_parroquia", "fecha_dia"])
    .size()
    .reset_index(name="conteo_llamadas_riesgo")
)

In [20]:
# Catálogo: asegurar 1 fila por cod_parroquia antes de merge
catalogo = pd.read_csv(
    os.path.join(ruta_catalogo, "catalogo_parroquias_ecuador.csv"),
    dtype={"cod_parroquia": str}
)

catalogo["cod_parroquia"] = catalogo["cod_parroquia"].astype(str).str.zfill(6)

# si hay repetidos, quédate con uno (o define una regla mejor si aplica)
catalogo = catalogo.drop_duplicates(subset=["cod_parroquia"], keep="first")

In [21]:
# Merge para lat/lon (ahora NO explota)
df_final = df_group.merge(catalogo[["cod_parroquia", "lat", "lon"]], on="cod_parroquia", how="left")


In [22]:
# Eliminar sin coords si quieres
df_final = df_final.dropna(subset=["lat", "lon"])

In [23]:
# Features temporales sobre fecha_dia
df_final["fecha_dia"] = pd.to_datetime(df_final["fecha_dia"])
df_final["mes"] = df_final["fecha_dia"].dt.month
df_final["dia"] = df_final["fecha_dia"].dt.day
df_final["dia_semana"] = df_final["fecha_dia"].dt.dayofweek

df_final.to_csv(os.path.join(ruta_dest, "ecu911_parroquia_dia_conteo.csv"), index=False)

print("ECU911 agregado por parroquia y día correctamente")
print("Registros finales:", len(df_final))

ECU911 agregado por parroquia y día correctamente
Registros finales: 138729
